In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import json

In [2]:
#data collected from https://www.cde.ca.gov/ds/sd/sd/filesabd.asp
#https://www.oregon.gov/ode/reports-and-data/students/Pages/Student-Enrollment-Reports.aspx

#text_file = "chronic_absentee_1819.txt"
#absence_data_df = pd.read_table(text_file, encoding="latin-1")
#absence_data_df.head(30)

url= "http://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=School&cYear=2019-20&cCat=Enrollment&cPage=filesenr.asp"

ca_enrollment_data = pd.read_table(url)
ca_enrollment_data


,CDS_CODE,COUNTY,DISTRICT,SCHOOL,ETHNIC,GENDER,KDGN,GR_1,GR_2,GR_3,...,GR_7,GR_8,UNGR_ELM,GR_9,GR_10,GR_11,GR_12,UNGR_SEC,ENR_TOTAL,ADULT
0,7617966004998,Contra Costa,West Contra Costa Unified,Tara Hills Elementary,2,M,3,3,3,1,...,0,0,0,0,0,0,0,0,14,0
1,7617966004998,Contra Costa,West Contra Costa Unified,Tara Hills Elementary,6,F,8,8,4,3,...,0,0,0,0,0,0,0,0,39,0
2,7617966004998,Contra Costa,West Contra Costa Unified,Tara Hills Elementary,4,F,1,1,1,1,...,0,0,0,0,0,0,0,0,12,0
3,7617966004998,Contra Costa,West Contra Costa Unified,Tara Hills Elementary,9,F,3,0,2,2,...,0,0,0,0,0,0,0,0,10,0
4,7617966004998,Contra Costa,West Contra Costa Unified,Tara Hills Elementary,7,M,8,4,1,3,...,0,0,0,0,0,0,0,0,26,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131611,37681303732625,San Diego,Grossmont Union High,Grossmont High,7,M,0,0,0,0,...,0,0,0,141,138,111,120,0,510,0
131612,37681890000001,San Diego,Lakeside Union Elementary,"Nonpublic, Nonsectarian Schools",5,M,0,0,0,0,...,0,1,0,0,0,0,0,0,2,0
131613,37681890000001,San Diego,Lakeside Union Elementary,"Nonpublic, Nonsectarian Schools",6,F,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
131614,37681890000001,San Diego,Lakeside Union Elementary,"Nonpublic, Nonsectarian Schools",7,F,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [3]:
#ca_enrollment_data.groupby("COUNTY").sum()
#modify demo to include ethnicity as columns
ca_enrollment_race = ca_enrollment_data.pivot_table(values='ENR_TOTAL', index='COUNTY', columns='ETHNIC', aggfunc='sum')
ca_enrollment_race.head()

ETHNIC,0,1,2,3,4,5,6,7,9
COUNTY,,,,,,,,,
Alameda,2410.0,597.0,60548.0,2276.0,10324.0,77433.0,20865.0,39186.0,13692.0
Alpine,1.0,36.0,NaN,NaN,NaN,2.0,NaN,26.0,5.0
Amador,32.0,99.0,20.0,8.0,20.0,925.0,20.0,2765.0,277.0
Butte,432.0,690.0,1777.0,100.0,124.0,7469.0,602.0,17002.0,1838.0
Calaveras,5.0,82.0,31.0,5.0,17.0,1128.0,39.0,3671.0,337.0


In [4]:
#rename the ethnicity columns
ca_enrollment_race_2 = ca_enrollment_race.rename(columns = {0:'OTHER',
                                    1:'AMERICAN_INDIAN',
                                    2:'ASIAN',
                                    3:'PACIFIC_ISLANDER',
                                    4:'FILIPINO',
                                    5:'LATINO',
                                    6:'AFRICAN_AMERICAN',
                                    7:'WHITE',
                                    9:'MULTI_RACE'})
ca_enrollment_race_2.reset_index(inplace=True)
ca_enrollment_race_2.head()

ETHNIC,COUNTY,OTHER,AMERICAN_INDIAN,ASIAN,PACIFIC_ISLANDER,FILIPINO,LATINO,AFRICAN_AMERICAN,WHITE,MULTI_RACE
0,Alameda,2410.0,597.0,60548.0,2276.0,10324.0,77433.0,20865.0,39186.0,13692.0
1,Alpine,1.0,36.0,NaN,NaN,NaN,2.0,NaN,26.0,5.0
2,Amador,32.0,99.0,20.0,8.0,20.0,925.0,20.0,2765.0,277.0
3,Butte,432.0,690.0,1777.0,100.0,124.0,7469.0,602.0,17002.0,1838.0
4,Calaveras,5.0,82.0,31.0,5.0,17.0,1128.0,39.0,3671.0,337.0


In [5]:
url= "https://www.oregon.gov/ode/reports-and-data/students/Documents/fallmembershipreport_20192020.xlsx"

or_enrollment_data = pd.read_excel(url, sheet_name = "District (19-20)")
or_enrollment_data
list(or_enrollment_data.columns)

['County',
 'Attending District Institution ID',
 'District Name',
 '2018-19 Total Enrollment',
 '2019-20 Total Enrollment',
 '2019-20 American Indian/Alaska Native ',
 '2019-20 % American Indian/Alaska Native ',
 '2019-20 Asian ',
 '2019-20 % Asian ',
 '2019-20 Native Hawaiian/ Pacific Islander',
 '2019-20 % Native Hawaiian/ Pacific Islander',
 '2019-20 Black/African American ',
 '2019-20 % Black/African American ',
 '2019-20 Hispanic/ Latino',
 '2019-20 % Hispanic/ Latino',
 '2019-20 White ',
 '2019-20 % White ',
 '2019-20 Multiracial ',
 '2019-20 % Multiracial ',
 '2019-20 Kindergarten',
 '2019-20 Grade One',
 '2019-20 Grade Two',
 '2019-20 Grade Three',
 '2019-20 Grade Four',
 '2019-20 Grade Five',
 '2019-20 Grade Six',
 '2019-20 Grade Seven',
 '2019-20 Grade Eight',
 '2019-20 Grade Nine',
 '2019-20 Grade Ten',
 '2019-20 Grade Eleven',
 '2019-20 Grade Twelve']

In [6]:
rds_connection_string = "postgres:postgres@localhost:5432/ELT_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
#Check for tables
engine.table_names()

[]

NameError: name 'clean_asian' is not defined

In [11]:
url = "https://data.wa.gov/resource/gtd3-scga.json"
print(requests.get(url))
print(requests.get(url).json())
washington_enrollment_data = requests.get(url).json()
print(json.dumps(washington_enrollment_data, indent=4, sort_keys=True))

<Response [200]>
[{'schoolyear': '2019-20', 'organizationlevel': 'State', 'county': 'Multiple', 'esdname': 'State Total', 'districtname': 'State Total', 'schoolname': 'State Total', 'gradelevel': '10th Grade', 'all_students': '84690', 'female': '41154', 'gender_x': '139', 'male': '43397', 'american_indian_alaskan_native': '1117', 'asian': '7082', 'black_african_american': '3680', 'hispanic_latino_of_any_race': '19862', 'native_hawaiian_other_pacific': '981', 'two_or_more_races': '6419', 'white': '45541', 'english_language_learners': '7594', 'foster_care': '427', 'homeless': '1854', 'low_income': '36377', 'migrant': '1789', 'military_parent': '1459', 'mobile': '2191', 'section_504': '4964', 'students_with_disabilities': '10161', 'non_english_language_learners': '77096', 'non_foster_care': '84263', 'non_homeless': '82836', 'non_low_income': '48313', 'non_migrant': '82901', 'non_military_parent': '83231', 'non_mobile': '82499', 'non_section_504': '79726', 'students_without_disabilities': 

[
    {
        "all_students": "84690",
        "american_indian_alaskan_native": "1117",
        "asian": "7082",
        "black_african_american": "3680",
        "county": "Multiple",
        "dataasof": "2020-02-11T00:00:00.000",
        "districtname": "State Total",
        "english_language_learners": "7594",
        "esdname": "State Total",
        "female": "41154",
        "foster_care": "427",
        "gender_x": "139",
        "gradelevel": "10th Grade",
        "hispanic_latino_of_any_race": "19862",
        "homeless": "1854",
        "low_income": "36377",
        "male": "43397",
        "migrant": "1789",
        "military_parent": "1459",
        "mobile": "2191",
        "native_hawaiian_other_pacific": "981",
        "non_english_language_learners": "77096",
        "non_foster_care": "84263",
        "non_homeless": "82836",
        "non_low_income": "48313",
        "non_migrant": "82901",
        "non_military_parent": "83231",
        "non_mobile": "82499",
  

In [12]:
washington_enrollment_data[0].keys()

dict_keys(['schoolyear', 'organizationlevel', 'county', 'esdname', 'districtname', 'schoolname', 'gradelevel', 'all_students', 'female', 'gender_x', 'male', 'american_indian_alaskan_native', 'asian', 'black_african_american', 'hispanic_latino_of_any_race', 'native_hawaiian_other_pacific', 'two_or_more_races', 'white', 'english_language_learners', 'foster_care', 'homeless', 'low_income', 'migrant', 'military_parent', 'mobile', 'section_504', 'students_with_disabilities', 'non_english_language_learners', 'non_foster_care', 'non_homeless', 'non_low_income', 'non_migrant', 'non_military_parent', 'non_mobile', 'non_section_504', 'students_without_disabilities', 'dataasof'])